# Tutorial for Real-Time fMRI Cloud-Based Framework

## Overview
This tutorial walks through an example using our cloud-based software framework for real-time fMRI studies, henceforth **rtcloud framework**. We have created a sample script (sample.py) in this notebook directory which builds a two-class classifier on a region of interest (ROI). This notebook will generate synthetic data for training and classification using the BrainIAK fmrisim_real_time_generator function.

We will begin wrapping the sample.py script within the rtcloud projectInterface. In a normal deployment the projectInterface (and sample.py) would then run in the cloud and the scanner images would be sent to the cloud where our sample.py script would build the classifier model and do classification. The projectInterface handles remote file communication for reading the scanner images and providing the classification results. It also has a web-based user interface for viewing and controlling the experiment and changing settings. In this notebook the different process all run on the the computer that jupyter is running on.

In the steps below, we will first start the projectInterface which will watch for dicom files to process. We will then start the synthetic data generator which will generate a new dicom image every 2 seconds. And finally we will start a web browser within this notebook for controlling the experiment. Once you start the web interface you can click the 'Run' button to start the sample.py script watching for and processing images via the projectInterface. (You can also open the web interface on a different tab in your browser, http://localhost:8888)


## Table of Contents

* [Before Running This Notebook](#setting_up)
* [Import Necessary Modules and Declare Important Variables](#import_modules)
* [Step 1: Start the ProjectInterface Web Server](#start_ProjectInterface)
* [Step 2: Start the Synthetic Data Generator](#start_DataGenerator)
* [Step 3: Open the Web Server on the localhost](#open_localhost)
* [Conclusion](#conclusion)

## Before Running This Notebook <a id='setting_up'></a>

Before you are able to run this notebook, you have to complete the installation instructions found in the accompanying [instructions README](https://github.com/brainiak/brainiak-aperture/blob/master/notebooks/real-time/README_INSTRUCTIONS.md). **Also, remember that you have to complete the following steps every time before you are able to run this notebook:**

1. Activate the conda environment for the rtcloud framework:
```
conda activate rtcloud
```
2. On the command line, create a global variable to the full path for the rtcloud framework repo. You must do this in order to use the functions we have created for the framework. And don't forget the forward slash "/" at the end.
```
export RTCLOUD_PATH=/PATH_TO_RTCLOUD/rt-cloud/
```
Double check that you did this correctly by typing the following command. 
```
ECHO $RTCLOUD_PATH
```
This should print the *full* path to the rtcloud framework folder.

## Import Necessary Modules and Declare Important Variables <a id='import_modules'></a>

In [1]:
import warnings; warnings.simplefilter('ignore')

#---- Import the necessary python modules

import os
import sys
import threading

#---- Declare and append important paths

# add the full path to the rtcloud repo
path_to_rtcloud = os.getenv('RTCLOUD_PATH')
if path_to_rtcloud == None:
    print("Please set RTCLOUD_PATH, see instructions")
    raise ValueError
# append the python path to the rtcloud repo to access rtCommon functions
sys.path.append(path_to_rtcloud)
    
# declare the path to this jupyter notebook
path_to_notebook = os.getcwd()  # check and change notebook path as needed

# declare the scripts that will be wrapped by the projectInterface and 
#   accessed through the webServer
scriptToRun = os.path.join(path_to_notebook, 'sample.py')
initScript = os.path.join(path_to_notebook, 'initialize.py')
finalizeScript = os.path.join(path_to_notebook, 'finalize.py')

#---- Load important rtcloud modules

from rtCommon.projectInterface import Web
from rtCommon.structDict import StructDict

#---- Load important brainiak modules
import brainiak.utils.fmrisim_real_time_generator as sim

#---- Declare the total number of TRs (timepoints) you want to generate, and how many should be training
num_TRs = 200
num_training = int(num_TRs*(2/3))

## Step 1: Start the ProjectInterface Web Server <a id='start_ProjectInterface'></a>

In [2]:
#---- Set up the config and parameters for this rtcloud tutorial

# NOTE: you can also change these parameters in the Settings tab on the web server
config = StructDict({
    'title' : 'rtCloud Tutorial',
    'sessionId' : '20200101T120000',
    'subjectName' : '001_synthetic',
    'datestr' : '20200101',
    
    'isSynthetic' : True,
    'numSynthetic' : num_TRs,
    'numTrainingTRs' : num_training,
    'imgDir' : '/tmp/notebook-simdata',
    
    'subjectNum' : 101,
    'subjectDay' : 1,
    'sessionNum' : 1,
    'runNum' : [1],
    'scanNum' : [14],

    # Plotting settings
    'plotTitle' : 'Realtime Plot',
    'plotXLabel' : 'TR #',
    'plotYLabel' : 'Classifier Prediction',
    'plotXRangeLow' : num_training,
    'plotXRangeHigh' : num_TRs,
    'plotYRangeLow' : -1,
    'plotYRangeHigh' : 1,
    'plotAutoRangeX' : False,
    'plotAutoRangeY' : True,

    # important variables about the dicom files
    'dicomNamePattern' : "rt_{run:03d}.dcm",
    'minExpectedDicomSize' : 200000,
})

params = StructDict({
    'fmriPyScript': scriptToRun,
    'initScript': initScript,
    'finalizeScript': finalizeScript,
    'filesremote': False,
    'port': 8889
})

#---- Start the web server

web = Web()
try:
    web.start(params, config, testMode=True)
    print(true)
except RuntimeError as err:
    # ignore event loop already running error
    if str(err) != 'This event loop is already running':
        raise

Listening on: http://localhost:8883


***

**IMPORTANT:**

You can only run this cell to start the projectInterface web server only once or else you will get the following runtime error: 
```
Web Server already running
```
If you want to re-start the web server, you have to first SHUT DOWN the kernel. Instructions for doing this can be found [here](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html#close-a-notebook-kernel-shut-down).

***

## Step 2: Start the Synthetic Data Generator <a id='start_DataGenerator'></a>

We will be using a BrainIAK function to create synthetic fMRI data. After you run the following cell, you can execute `sim_settings` in a separate cell to take a look at the different parameter settings. For instance, you will find that the synthetic data is being produced at a rate of 2 seconds per TRs.

In [3]:
#---- Set up the parameters for the synthetic data generator

sim_settings = sim.default_settings
sim_settings['save_dicom'] = True
sim_settings['save_realtime'] = True
sim_settings['numTRs'] = num_TRs
sim_settings['different_ROIs'] = True

outdir = '/tmp/notebook-simdata'
if not os.path.exists(outdir):
    os.makedirs(outdir)

#---- Run sim.generate_data(outdir, sim_settings) as a thread

syndata_thread = threading.Thread(name='syndata', 
                                  target=sim.generate_data, 
                                  args=(outdir, sim_settings))
syndata_thread.setDaemon(True)
syndata_thread.start()

***
**IMPORTANT:**

You can only run this cell to generate the synthetic data in "real-time" once unless you delete the data that you have already created. Otherwise, all the data (which is found in the `/tmp/notebook-simdata` folder) will be automatically available when you open the web server, instead of having the File Watcher wait for incoming data one TR at a time.

***

## Step 3: Open the Web Server on the localhost <a id='open_localhost'></a>

The first time you open the web server, either in the cell below or in a separate browser tab, you will be prompted to enter a username and password. In this demonstration, you will use "test" for both. Once you have logged in you will be on the Run tab. You can start the sample.py script running and waiting for dicom data to process by clicking the 'run' button within the webpage 'Run' tab. Then you will see the output as the script progresses within the view pane of the Run tab.

If you want to re-run the sample.py script after making changes, you will most likely get the following error message: `Error: Client thread already runnning, skipping new request`. You will have to restart the kernel like in step 1.

In [4]:
%%html
<iframe src="http://localhost:8889" width="800" height="600"></iframe>

## Conclusion <a id='conclusion'></a>

You have now completed running a real-time experiment using a distributed computing pipeline (managed by the projectInterface). The synthetic data was sent to the sample.py script in real-time as it was generated, and the script started by building a classifier and then performing classification.

As a next step you can try modifying or using your own script in place of sample.py, and you can also try running the projectInterface on a cloud VM which eliminates the need for expensive computer hardware within the control room.

We're excited to see what interesting real-time experiments you will create!
